# Hubot インストール

# はじめに

本書ではHubot環境の構築手順です。他ツールとの連携に関しては別紙で扱います。
また本書作業についてはJenkinsをインストールしたVM内で作業してください。
terminalから以下のコマンドでログインできます。
```
IP="$(< ~/vdc_host_ip)"
PORT="$(< ~/vdc_instance_port)"
ssh -qi ~/mykeypair -p $PORT root@$IP
```

# 動作環境

CentOS-6.6で動作環境を行いました。それよりも古い環境の場合は動作確認してませんので、ご了承下さい。

# Hubotのインストール

HubotはCoffeeScriptで書かれているので、CoffeeScript実行環境を構築します。npmコマンドを利用してインストールします。

またnpmコマンドをインストールするための依存パッケージも合わせてインストールします。

## Software collectionsリポジトリの追加

### centos-release-scl, centos-release-scl-rhのインストール

```
# yum install centos-release-scl-rh centos-release-scl
```

実行結果例：
>```
> # yum install centos-release-scl-rh centos-release-scl
> Loaded plugins: fastestmirror
> Setting up Install Process
> Loading mirror speeds from cached hostfile
>  * base: www.ftp.ne.jp
>  * epel: ftp.riken.jp
>  * extras: www.ftp.ne.jp
>  * updates: www.ftp.ne.jp
> Resolving Dependencies
> --> Running transaction check
> ---> Package centos-release-scl.noarch 10:7-3.el6.centos will be installed
> ---> Package centos-release-scl-rh.noarch 0:2-3.el6.centos will be installed
> --> Finished Dependency Resolution
> 
> Dependencies Resolved
> 
>=============================================================================================>=========================
> Package                             Arch                 Version                          Repository            Size
>=============================================================================================>=========================
>Installing:
> centos-release-scl                  noarch               10:7-3.el6.centos                extras                12 k
> centos-release-scl-rh               noarch               2-3.el6.centos                   extras                12 k
>
>Transaction Summary
>=============================================================================================>=========================
>Install       2 Package(s)
>
>Total download size: 24 k
>Installed size: 39 k
>Is this ok [y/N]: y
>Downloading Packages:
>(1/2): centos-release-scl-7-3.el6.centos.noarch.rpm                                            |  12 kB     00:00
>(2/2): centos-release-scl-rh-2-3.el6.centos.noarch.rpm                                         |  12 kB     00:00
>----------------------------------------------------------------------------------------------------------------------
>Total                                                                                 216 kB/s |  24 kB     00:00
>Running rpm_check_debug
>Running Transaction Test
>Transaction Test Succeeded
>Running Transaction
>  Installing : centos-release-scl-rh-2-3.el6.centos.noarch                                                        1/2
>  Installing : 10:centos-release-scl-7-3.el6.centos.noarch                                                        2/2
>  Verifying  : centos-release-scl-rh-2-3.el6.centos.noarch                                                        1/2
>  Verifying  : 10:centos-release-scl-7-3.el6.centos.noarch                                                        2/2
>
>Installed:
>  centos-release-scl.noarch 10:7-3.el6.centos              centos-release-scl-rh.noarch 0:2-3.el6.centos
>
>Complete!
>```

### リポジトリリストの確認

Software collectionsリポジトリが有効になった事を確認します。

```
# yum repolist
```

実行結果例：

> ```
> # yum repolist
> Loaded plugins: fastestmirror
> Loading mirror speeds from cached hostfile
>  * base: ftp.nara.wide.ad.jp
>  * extras: www.ftp.ne.jp
>  * updates: ftp.nara.wide.ad.jp
> repo id          repo name                                                status
> base             CentOS-6.6 - Base                                         6,518
> centos-sclo-rh CentOS-6 - SCLo rh                                        5,741
> centos-sclo-sclo CentOS-6 - SCLo sclo                                   323
> epel             Extra Packages for Enterprise Linux 6 - x86_64           11,371
> extras           CentOS-6.6 - Extras                                          37
> updates          CentOS-6.6 - Updates                                        965
> repolist: 18,891
> ```

`centos-sclo-rh`、`centos-sclo-sclo`が表示されていれば、追加したリポジトリは有効です。

## nodejsをインストールするのに必要な依存パッケージのインストール

nodejsをインストールするためにgccとpythonをを更新します。

```
# yum install devtoolset-3-gcc python27
```

実行結果例:
> ```
> Loaded plugins: fastestmirror
> Setting up Install Process
> Loading mirror speeds from cached hostfile
> * base: www.ftp.ne.jp
> * epel: ftp.riken.jp
> * extras: www.ftp.ne.jp
> * updates: www.ftp.ne.jp
> Resolving Dependencies
> --> Running transaction check
> ---> Package devtoolset-3-gcc.x86_64 0:4.9.2-6.2.el6 will be installed
> ...(省略)...
> Complete!
> ```

## PATHの設定

インストールしたパッケージは/opt/rh配下にインストールされるため別途PATHを通す必要があります。

```
# source /opt/rh/devtoolset-3/enable
# source /opt/rh/python27/enable
```

実行結果例:

> ```
> # gcc --version
> gcc (GCC) 4.9.2 20150212 (Red Hat 4.9.2-6)
> Copyright (C) 2014 Free Software Foundation, Inc.
> This is free software; see the source for copying conditions.  There is NO
> warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
> # python -V
> Python 2.7.13
```

## nodejsリポジトリの追加

nodejsをインストールするためにnodejsリポジトリを追加します。

```
# curl -sL https://rpm.nodesource.com/setup_8.x | bash -
```

実行結果例:

> ```
> ## Installing the NodeSource Node.js 8.x repo...
>
>
> ## Inspecting system...
>
> + rpm -q --whatprovides redhat-release || rpm -q --whatprovides centos-release || rpm -q --whatprovides cloudlinux-rel
> ease || rpm -q --whatprovides sl-release
> + uname -m
>
> ## Confirming "el6-x86_64" is supported...
>
> + curl -sLf -o /dev/null 'https://rpm.nodesource.com/pub_8.x/el/6/x86_64/nodesource-release-el6-1.noarch.rpm'
>
> ## Downloading release setup RPM...
>
> + mktemp
> + curl -sL -o '/tmp/tmp.iwbdJp8Liu' 'https://rpm.nodesource.com/pub_8.x/el/6/x86_64/nodesource-release-el6-1.noarch.rp
> m'
>
> ## Installing release setup RPM...
>
> + rpm -i --nosignature --force '/tmp/tmp.iwbdJp8Liu'
>
> ## Cleaning up...
>
> + rm -f '/tmp/tmp.iwbdJp8Liu'
>
> ## Checking for existing installations...
>
> + rpm -qa 'node|npm' | grep -v nodesource
>
> ## Run `yum install -y nodejs` (as root) to install Node.js 8.x and npm.
> ## You may also need development tools to build native addons:
> ##   `yum install -y gcc-c++ make`
> ```

## nodejsのインストール

nodejsをインストールします。それと、Hubotが依存するパケージもインストールします。

```
# yum install -y nodejs redis libicu-devel
```

実行結果例：

> ```
> # yum install -y nodejs redis libicu-devel
Loaded plugins: fastestmirror
Setting up Install Process
Loading mirror speeds from cached hostfile
 * base: ftp.nara.wide.ad.jp
 * extras: www.ftp.ne.jp
 * updates: ftp.nara.wide.ad.jp
Resolving Dependencies
--> Running transaction check
---> Package libicu-devel.x86_64 0:4.2.1-9.1.el6_2 will be installed
> ...(省略)...
> Complete!
> ```

ネットワーク帯域に依存しますが、1～2分で完了します。

## npmによるインストール

### npm-install-missingのインストール

npm で使用する足りないモジュールを一括インストールするために使用します。

```
# npm install -g npm-install-missing
```

実行結果例:

> ```
> # npm install -g npm-install-missing
> /usr/bin/npm-install-missing -> /usr/lib/node_modules/npm-install-missing/bin/npm-install-missing
> + npm-install-missing@0.1.4
> added 414 packages in 15.213s
>```

実行して足りないものがないか確認

```
# npm-install-missing
```

実行結果例:

> ```
> # npm-install-missing: No modules seem to be missing.  Huzzah!
> ```

### Hubotのインストール

hubot関連パッケージは、nodejsパッケージ専用ツール`npm`でインストールします。

```
# npm install -g coffee-script hubot
```

実行結果例：

> ```
> # npm install -g coffee-script hubot
> npm WARN deprecated connect@2.30.2: connect 2.x series is deprecated
> /usr/bin/coffee -> /usr/lib/node_modules/coffee-script/bin/coffee
> /usr/bin/cake -> /usr/lib/node_modules/coffee-script/bin/cake
> /usr/bin/hubot -> /usr/lib/node_modules/hubot/bin/hubot
> + coffee-script@1.12.7
> + hubot@2.19.0
> added 135 packages and updated 1 package in 10.195s
> ```

ネットワーク帯域に依存しますが、3～4分で完了します。

### プロジェクトセットアップツールのインストール

Hubotプロジェクトをセットアップする為のツールをインストールします。

```
# npm install -g yo generator-hubot
```

> ```
> # npm install -g yo generator-hubot
> npm WARN deprecated CSSselect@0.4.1: the module is now available as 'css-select'
> npm WARN deprecated CSSwhat@0.4.7: the module is now available as 'css-what'
> npm WARN deprecated minimatch@2.0.10: Please update to minimatch 3.0.2 or higher to avoid a RegExp DoS issue
> npm WARN deprecated minimatch@0.3.0: Please update to minimatch 3.0.2 or higher to avoid a RegExp DoS issue
> /usr/bin/yo -> /usr/lib/node_modules/yo/lib/cli.js
> /usr/bin/yo-complete -> /usr/lib/node_modules/yo/lib/completion/index.js
> ...(省略)...
> Everything looks all right!
> + yo@2.0.0
> + generator-hubot@0.4.0
> added 681 packages in 41.165s
> ```

ネットワーク帯域に依存しますが、10分前後で完了します。